# AIAP 20 - Technical Assessment
### Task 1 - Exploratory Data Analysis (EDA)

## Step 1: Import Libraries

In [ ]:
import sqlite3
import pandas as pd


## Step 2: Import Database and convert to DataFrame

In [28]:
con = sqlite3.connect("data/bmarket.db") # connect to SQLite database
curr = con.cursor()

Reference: docs.python.org/3/library/sqlite3.html

### Get The Table Name

In [29]:
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables = curr.fetchall()

print (tables) # check how many tables are there in the db.

def table_name(): # function used to assign table name
    for table in tables:
        return table[0]

print(table_name())


[('bank_marketing',)]
bank_marketing


### Convert .db to DataFrame(pd)

In [30]:
dataset_raw = pd.read_sql_query(f"SELECT * FROM {table_name()}", con)

## Step 3: Dataset Overview

### Check basic info

In [31]:
# Check the number of Rows and Columns
dataset_raw.shape

(41188, 12)

In [32]:
# Check the Header
dataset_raw.head()

,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status
0,32885,57 years,technician,married,high.school,no,no,yes,Cell,1,999,no
1,3170,55 years,unknown,married,unknown,unknown,yes,no,telephone,2,999,no
2,32207,33 years,blue-collar,married,basic.9y,no,no,no,cellular,1,999,no
3,9404,36 years,admin.,married,high.school,no,no,no,Telephone,4,999,no
4,14021,27 years,housemaid,married,high.school,no,None,no,Cell,2,999,no


In [33]:
# Check basic info of the dataset
dataset_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Client ID              41188 non-null  int64 
 1   Age                    41188 non-null  object
 2   Occupation             41188 non-null  object
 3   Marital Status         41188 non-null  object
 4   Education Level        41188 non-null  object
 5   Credit Default         41188 non-null  object
 6   Housing Loan           16399 non-null  object
 7   Personal Loan          37042 non-null  object
 8   Contact Method         41188 non-null  object
 9   Campaign Calls         41188 non-null  int64 
 10  Previous Contact Days  41188 non-null  int64 
 11  Subscription Status    41188 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.8+ MB


In [34]:

# Checking for missing values
dataset_raw.isnull().sum()

Client ID                    0
Age                          0
Occupation                   0
Marital Status               0
Education Level              0
Credit Default               0
Housing Loan             24789
Personal Loan             4146
Contact Method               0
Campaign Calls               0
Previous Contact Days        0
Subscription Status          0
dtype: int64

In [35]:
dataset_raw["Housing Loan"].unique()

array(['no', 'yes', None, 'unknown'], dtype=object)

In [36]:
dataset_raw["Housing Loan"].dropna().unique()

array(['no', 'yes', 'unknown'], dtype=object)

In [37]:
dataset_raw["Personal Loan"].unique()

array(['yes', 'no', None, 'unknown'], dtype=object)

In [38]:
dataset_raw["Housing Loan"].dropna().unique()

array(['no', 'yes', 'unknown'], dtype=object)

In [39]:

# Describe the dataset for numerical features
dataset_raw.describe()

,Client ID,Campaign Calls,Previous Contact Days
count,41188.00000,41188.000000,41188.000000
mean,20594.50000,2.051374,962.475454
std,11890.09578,3.171345,186.910907
min,1.00000,-41.000000,0.000000
25%,10297.75000,1.000000,999.000000
50%,20594.50000,2.000000,999.000000
75%,30891.25000,3.000000,999.000000
max,41188.00000,56.000000,999.000000


In [50]:
filtered_data = dataset_raw[dataset_raw["Previous Contact Days"] != 999]
filtered_data.describe()
# filtered_data["Previous Contact Days"].describe()

,Client ID,Campaign Calls,Previous Contact Days
count,1515.000000,1515.000000,1515.000000
mean,37871.124092,1.411881,6.014521
std,3477.361721,1.694070,3.824906
min,24109.000000,-7.000000,0.000000
25%,37281.000000,1.000000,3.000000
50%,39009.000000,1.000000,6.000000
75%,40163.000000,2.000000,7.000000
max,41183.000000,13.000000,27.000000


In [40]:
# Check for duplicate rows
int(dataset_raw.duplicated().sum())

0